In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

c:\Users\milan\miniconda3\envs\masonai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to("cuda")

In [3]:
def preprocess_function(examples):
  inputs = examples["Name"] + ". " + examples["Description"] if examples["Description"] else ""
  targets = examples["Serialized"]
  model_inputs = tokenizer(inputs, text_target=targets)

  return model_inputs

dataset = load_dataset("csv", data_files="clean-data/20240428-rbxldata.csv", delimiter=",")
dataset["train"] = dataset["train"].select(range(1000))
tokenized_data = dataset.map(preprocess_function, batched=False)

In [4]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [5]:
training_args = Seq2SeqTrainingArguments(
  output_dir="mason-t5",
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=8,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=24,
  predict_with_generate=True,
  fp16=True,
)

In [6]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_data["train"],
  tokenizer=tokenizer,
  data_collator=data_collator,
)

In [7]:
trainer.train()
trainer.save_model("mason-t5-4-28")

  0%|          | 0/3000 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 47.48 GiB. GPU 